In [ ]:
import os
import pandas as pd
import numpy as np

from skimage import io
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from torchvision import models

from tqdm.notebook import tqdm, trange

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
TEST_IMG = "../input/petfinder-pawpularity-score/test"
TRAIN_IMG = "../input/petfinder-pawpularity-score/train"
TEST_CSV = "../input/petfinder-pawpularity-score/test.csv"
TRAIN_CSV = "../input/petfinder-pawpularity-score/train.csv"
SUB = "../input/petfinder-pawpularity-score/sample_submission.csv"

train_csv = pd.read_csv(TRAIN_CSV)
test_csv = pd.read_csv(TEST_CSV)
sub_csv = pd.read_csv(SUB)

In [ ]:
# image transform
basic_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# get pretrained VGG16
model_vgg16 = models.vgg16()
model_vgg16.load_state_dict(torch.load('../input/pretrained-vgg16/vgg16.pt'))

In [ ]:
# get avgpool layer result and use it as a additional features for next model
def avgpool_features(model, img_dir, csv_file, data, transform):
    
    avg_features = []
    
    for img in tqdm(csv_file['Id'].values, desc=data):
        
        img_path = os.path.join(img_dir, img+'.jpg')
        image = io.imread(img_path)
        image = transform(Image.fromarray(image)).unsqueeze(dim=0)
        
        result = model.avgpool(image)
        avg_features.append(torch.flatten(result, 1).numpy()[0])
        
    avg_features_columns = ['af_'+str(i) for i in range(len(avg_features[0]))]
    avg_pool_df = pd.DataFrame(avg_features, columns=avg_features_columns)
    
    df = csv_file.join(avg_pool_df)
    df.columns = df.columns.str.lower()
    df = df.rename({
        'subject focus': 'focus'
    }, axis=1)
    
    return df

In [ ]:
train_data = avgpool_features(model_vgg16, TRAIN_IMG, train_csv, 'Train', basic_transform)
test_data = avgpool_features(model_vgg16, TEST_IMG, test_csv, 'Test', basic_transform)

In [ ]:
features = test_data.drop(['id'], axis=1).columns.values

In [ ]:
def preprocess_data(data, batch_size, num_workers=0, train=True, ft=features):
    
    if train:
        
        data = data.sample(frac=1, random_state=17)
        
        val, train = data[:200], data[200:]
        
        train_target = torch.Tensor(train['pawpularity'].values).to(device)
        train_features = torch.Tensor(train.drop(['pawpularity'], axis=1)[ft].values).to(device)
        
        val_target = torch.Tensor(val['pawpularity'].values).to(device)
        val_features = torch.Tensor(val.drop(['pawpularity'], axis=1)[ft].values).to(device)

        train_loader = DataLoader(
            TensorDataset(train_features,train_target), 
            shuffle=True, 
            batch_size=batch_size,
            num_workers=num_workers
        )
        val_loader = DataLoader(
            TensorDataset(val_features,val_target), 
            shuffle=True, 
            batch_size=batch_size,
            num_workers=num_workers
        )
        
        return train_loader, val_loader
        
    test_features = torch.Tensor(data[ft].values).to(device)
    
    test_loader = DataLoader(
        TensorDataset(test_features), 
        shuffle=False, 
        batch_size=batch_size,
        num_workers=num_workers
    )
    
    return test_loader

In [ ]:
train_loader, val_loader = preprocess_data(train_data, 50, 0, True, features)
test_loader = preprocess_data(test_data, 50, 0,  False, features)
loaders = {
    'train': train_loader, 
    'valid': val_loader, 
    'test': test_loader
}

In [ ]:
len(features)

In [ ]:
criterion = nn.MSELoss()

def get_optimizer_scratch(model):
    return optim.Adam(model.parameters(), lr = 0.00002)

In [ ]:
class EpicNet(nn.Module):
    def __init__(self):
        super(EpicNet, self).__init__()
        self.l1  = nn.Linear(159, 256)
        self.l2  = nn.Linear(256, 128)
        self.l3  = nn.Linear(128, 64)
        self.l4  = nn.Linear(64, 64)
        self.l5  = nn.Linear(64, 1)
        self.dropout = nn.Dropout(p=0.3)
        self.batch_norm1 = nn.BatchNorm1d(256)

    def forward(self, x):
        x = self.l1(x)
        x = self.dropout(x)
        x = self.batch_norm1(x)
        x = F.relu(self.l2(x))
        x = self.dropout(x)
        x = F.relu(self.l3(x))
        x = self.dropout(x)
        x = F.silu(self.l4(x))
        x = self.dropout(x)
        x = self.l5(x)

        return x

model_seq= EpicNet()
print(model_seq)

In [ ]:
def train(n_epochs, loaders, model, optimizer, criterion, device, save_path):

    valid_loss_min = np.Inf 
    model.to(device)
    
    for epoch in range(1, n_epochs+1):

        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            
            data.to(device)
            target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += ((1/(batch_idx+1))*(loss.data.item()-train_loss))

        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):

            data.to(device)
            target.to(device)

            with torch.no_grad():
                output = model(data)
                loss = criterion(output, target.unsqueeze(1))
                valid_loss += ((1/(1+batch_idx+1))*(loss.data.item()-valid_loss))
            
        if epoch % 10  == 0:
            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tValid RMSE: {:.6f}'.format(
                epoch, 
                train_loss,
                valid_loss,
                np.sqrt(valid_loss)
                ))
            
        if valid_loss <= valid_loss_min:
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        
    return model

In [ ]:
def custom_weight_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y,y)
        m.bias.data.fill_(0)
    
model_seq.apply(custom_weight_init)
model_final = train(300, loaders, model_seq, get_optimizer_scratch(model_seq), criterion, device, 'model_seq.pt')


In [ ]:
model_final.load_state_dict(torch.load('model_seq.pt'))

In [ ]:
out = []
model_final.eval()

for batch_idx, data in enumerate(loaders['test']):
    out += model_final(data[0]).view(-1).tolist()

In [ ]:
sub_csv['Pawpularity'] = pd.Series(out)
sub_csv.to_csv('sample_submission.csv', index=False)